<a href="https://colab.research.google.com/github/fredericmenezes/estudo_google_or-tools/blob/main/job_shop_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# job_shop_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/fredericmenezes/estudo_google_or-tools/blob/main/job_shop_example.ipynb"><img src="https://raw.githubusercontent.com/fredericmenezes/estudo_google_or-tools/main/img/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/fredericmenezes/estudo_google_or-tools/blob/main/job_shop_example.ipynb"><img src="https://raw.githubusercontent.com/fredericmenezes/estudo_google_or-tools/main/img/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [2]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


To access the template that is stored in GitHub repository, you can clone it inside the project_folder using the following:

In [5]:
!git clone https://github.com/fredericmenezes/estudo_google_or-tools.git

Cloning into 'estudo_google_or-tools'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 21 (delta 4), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (21/21), 4.38 KiB | 1.09 MiB/s, done.


Generic job shop function (accepts any size _m x n_ Matrix).

In [12]:
import collections
import datetime
from ortools.sat.python import cp_model

def jobshop_scheduler(jobs_data):
    # Create the model.
    model = cp_model.CpModel()

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)

    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.Value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1]))

        # Create per machine output lines.
        date = datetime.datetime(2023, 2, 28, 1, 0, 0)
        output = []
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_%i' % (assigned_task.job, assigned_task.index)

                start = date + datetime.timedelta(minutes=assigned_task.start)
                finish = date + datetime.timedelta(minutes=assigned_task.start + assigned_task.duration)

                start_minutes = start - date
                finish_minutes = finish - date

                start_min = int(start_minutes.total_seconds() / 60)
                finish_min = int(finish_minutes.total_seconds() / 60)

                output.append(dict(Task='Machine ' + str(machine), Start=start_min, Finish=finish_min, Name=name,
                                   Job='Job_' + str(assigned_task.job)))

        # Finally print the solution found.
        print('Optimal Schedule Length: %i' % solver.ObjectiveValue())
        print(output)

    return output

Minimal jobshop example.

In [13]:
if __name__ == '__main__':

    # Data.
    jobs_data = [  # task = (machine_id, processing_time).
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)]  # Job2
    ]

    jobshop_scheduler(jobs_data)

Optimal Schedule Length: 11
[{'Task': 'Machine 0', 'Start': 0, 'Finish': 2, 'Name': 'job_1_0', 'Job': 'Job_1'}, {'Task': 'Machine 0', 'Start': 2, 'Finish': 5, 'Name': 'job_0_0', 'Job': 'Job_0'}, {'Task': 'Machine 1', 'Start': 0, 'Finish': 4, 'Name': 'job_2_0', 'Job': 'Job_2'}, {'Task': 'Machine 1', 'Start': 5, 'Finish': 7, 'Name': 'job_0_1', 'Job': 'Job_0'}, {'Task': 'Machine 1', 'Start': 7, 'Finish': 11, 'Name': 'job_1_2', 'Job': 'Job_1'}, {'Task': 'Machine 2', 'Start': 2, 'Finish': 3, 'Name': 'job_1_1', 'Job': 'Job_1'}, {'Task': 'Machine 2', 'Start': 4, 'Finish': 7, 'Name': 'job_2_1', 'Job': 'Job_2'}, {'Task': 'Machine 2', 'Start': 7, 'Finish': 9, 'Name': 'job_0_2', 'Job': 'Job_0'}]


Another example using csv templates.

In [14]:
import csv

def is_number(string):
    try:
        int(string)
        return True
    except ValueError:
        return False

def get_data(path):
    try:
        with open(path, "r") as file:
            reader = csv.reader(file)
            next(reader)
            # TODO: Read content
            global jobs_data
            jobs_data = []
            for row in reader:
                job = []
                for i in range(1, len(row) - 1):
                    if is_number(row[i]):
                        if i % 2 == 1:
                            job.append((int(row[i]), int(row[i + 1])))
                # print(job)
                jobs_data.append(job)
    except:
        print("Error! Check the template file")
        return 1
    return 0



if __name__ == '__main__':
    get_data("/content/estudo_google_or-tools/template/template_29job.csv")
    jobshop_scheduler(jobs_data)

Optimal Schedule Length: 139
[{'Task': 'Machine 0', 'Start': 0, 'Finish': 12, 'Name': 'job_22_0', 'Job': 'Job_22'}, {'Task': 'Machine 0', 'Start': 12, 'Finish': 14, 'Name': 'job_18_0', 'Job': 'Job_18'}, {'Task': 'Machine 0', 'Start': 14, 'Finish': 15, 'Name': 'job_23_2', 'Job': 'Job_23'}, {'Task': 'Machine 0', 'Start': 15, 'Finish': 27, 'Name': 'job_21_0', 'Job': 'Job_21'}, {'Task': 'Machine 0', 'Start': 27, 'Finish': 39, 'Name': 'job_7_1', 'Job': 'Job_7'}, {'Task': 'Machine 0', 'Start': 39, 'Finish': 51, 'Name': 'job_17_0', 'Job': 'Job_17'}, {'Task': 'Machine 0', 'Start': 51, 'Finish': 63, 'Name': 'job_18_1', 'Job': 'Job_18'}, {'Task': 'Machine 0', 'Start': 63, 'Finish': 64, 'Name': 'job_10_1', 'Job': 'Job_10'}, {'Task': 'Machine 0', 'Start': 64, 'Finish': 65, 'Name': 'job_15_1', 'Job': 'Job_15'}, {'Task': 'Machine 0', 'Start': 65, 'Finish': 67, 'Name': 'job_1_0', 'Job': 'Job_1'}, {'Task': 'Machine 0', 'Start': 67, 'Finish': 69, 'Name': 'job_5_0', 'Job': 'Job_5'}, {'Task': 'Machine 0'